# Lab Exam1

## Case Study 1:Consider  attached air quality data (named Real_Combine.csv) file with following feature values:
T == Average Temperature (°C)
TM == Maximum temperature (°C)
Tm == Minimum temperature (°C)
SLP == Atmospheric pressure at sea level (hPa)
H == Average relative humidity (%)
VV == Average visibility (Km)
V == Average wind speed (Km/h)
VM == Maximum sustained wind speed (Km/h)
PM2.5== Fine particulate matter (PM2.5) is an air pollutant that is a concern for people's health when levels in air are high
## Build a DNN model which will give you optimum MAE(mean absolute error) value on test data.
## (Note:Use hyper parameter tuning on followings:
HyperparametersHow many number of hidden layers we should have(take range 2-20)?How many number of neurons we should have in hidden layers(range(32,512,32))?
different optimizer.

In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

!pip install keras_tuner
import keras_tuner as kt

import warnings
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
df=pd.read_csv('/content/Meet Agnihotri - Real_Combine.csv')
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1093 entries, 0 to 1092
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   T       1093 non-null   float64
 1   TM      1093 non-null   float64
 2   Tm      1093 non-null   float64
 3   SLP     1093 non-null   float64
 4   H       1093 non-null   float64
 5   VV      1093 non-null   float64
 6   V       1093 non-null   float64
 7   VM      1093 non-null   float64
 8   PM 2.5  1092 non-null   float64
dtypes: float64(9)
memory usage: 77.0 KB


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
T,1093.0,26.009241,7.237401,6.7,19.300000,28.200000,31.700000,38.5
TM,1093.0,32.482251,6.679078,9.8,27.800000,34.200000,37.000000,45.5
Tm,1093.0,19.460201,7.438653,0.0,12.100000,21.200000,26.000000,32.7
SLP,1093.0,1008.081885,7.529237,991.5,1001.100000,1008.100000,1015.000000,1023.2
H,1093.0,62.918573,15.709816,20.0,54.000000,64.000000,74.000000,98.0
VV,1093.0,2.003111,0.747541,0.3,1.600000,1.900000,2.600000,5.8
V,1093.0,6.751510,3.841137,0.4,3.700000,6.500000,9.100000,24.4
VM,1093.0,15.805124,7.308435,1.9,11.100000,14.800000,18.300000,57.6
PM 2.5,1092.0,109.090984,84.465790,0.0,41.833333,83.458333,158.291667,404.5


In [5]:
df.isnull().sum()

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    1
dtype: int64

In [6]:
df.shape

(1093, 9)

In [7]:
df = df.dropna(how='any',axis=0)

In [8]:
df.shape

(1092, 9)

In [9]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [10]:
X[:5]

,T,TM,Tm,SLP,H,VV,V,VM
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2


In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_X = scaler.fit_transform(X)

In [12]:
scaled_X[:5]

array([[0.02201258, 0.        , 0.14678899, 0.82334385, 0.93589744,
        0.03636364, 0.1625    , 0.13464991],
       [0.03459119, 0.08123249, 0.13455657, 0.85173502, 0.85897436,
        0.05454545, 0.16666667, 0.16517056],
       [0.        , 0.10084034, 0.0733945 , 0.88012618, 0.79487179,
        0.05454545, 0.18333333, 0.16517056],
       [0.05974843, 0.15966387, 0.10091743, 0.85804416, 0.66666667,
        0.09090909, 0.32083333, 0.33572711],
       [0.17924528, 0.31092437, 0.13455657, 0.81388013, 0.52564103,
        0.18181818, 0.34583333, 0.36445242]])

In [13]:
scaled_X.shape

(1092, 8)

In [14]:
y.shape

(1092,)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.3, random_state=100)

In [16]:
X_train.shape

(764, 8)

In [17]:
X_test.shape

(328, 8)

Finding the optimum hyperparameters

In [18]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    optimizer=hp.Choice("optimizer",values=['adam','rmsprop','adadelta','sgd','nadam'])
    model.compile(
        optimizer=optimizer,
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [19]:
tuner = kt.RandomSearch(
    build_model,
    objective=kt.Objective("val_mean_absolute_error", direction="min"),
    max_trials=5,
    executions_per_trial=5)

In [20]:
tuner.search(X_train, y_train,
             epochs=10,
             validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 09s]
val_mean_absolute_error: 48.26458969116211

Best val_mean_absolute_error So Far: 40.46221694946289
Total elapsed time: 00h 01m 18s


In [21]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 6
units_0: 352
units_1: 160
optimizer: nadam
units_2: 224
units_3: 32
units_4: 320
units_5: 128
units_6: 224
units_7: 128
units_8: 448
units_9: 192
units_10: 384
units_11: 288
units_12: 448
units_13: 96
units_14: 480
units_15: 352
units_16: 384
units_17: 448
Score: 40.46221694946289
Trial summary
Hyperparameters:
num_layers: 4
units_0: 288
units_1: 256
optimizer: nadam
units_2: 96
units_3: 416
units_4: 160
units_5: 480
units_6: 448
units_7: 192
units_8: 192
units_9: 160
units_10: 320
units_11: 352
units_12: 480
units_13: 224
units_14: 192
units_15: 480
units_16: 416
units_17: 64
Score: 40.775115203857425
Trial summary
Hyperparameters:
num_layers: 2
units_0: 352
units_1: 288
optimizer: adam
units_2: 480
units_3: 512
units_4: 160
units_5: 256
units_6: 64
units_7: 224
units_8: 32
units_9: 384
units_10: 96
units_11: 32
units_12: 288
units_13: 512
units_14: 64
units_15: 64
units_1

In [22]:
tuned_model=tuner.get_best_models(num_models=1)[0]

In [23]:
tuned_model.fit(X_train, y_train, epochs=50)

Epoch 1/50
24/24 [==============================] - 1s 11ms/step - loss: 39.5266 - mean_absolute_error: 39.5266
Epoch 2/50
24/24 [==============================] - 0s 5ms/step - loss: 39.1010 - mean_absolute_error: 39.1010
Epoch 3/50
24/24 [==============================] - 0s 5ms/step - loss: 40.4755 - mean_absolute_error: 40.4755
Epoch 4/50
24/24 [==============================] - 0s 5ms/step - loss: 38.4458 - mean_absolute_error: 38.4458
Epoch 5/50
24/24 [==============================] - 0s 4ms/step - loss: 39.1773 - mean_absolute_error: 39.1773
Epoch 6/50
24/24 [==============================] - 0s 5ms/step - loss: 38.5095 - mean_absolute_error: 38.5095
Epoch 7/50
24/24 [==============================] - 0s 4ms/step - loss: 37.8827 - mean_absolute_error: 37.8827
Epoch 8/50
24/24 [==============================] - 0s 4ms/step - loss: 38.5434 - mean_absolute_error: 38.5434
Epoch 9/50
24/24 [==============================] - 0s 7ms/step - loss: 37.6300 - mean_absolute_error: 37.6300


In [24]:
tuned_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 352)               3168      
                                                                 
 dense_1 (Dense)             (None, 160)               56480     
                                                                 
 dense_2 (Dense)             (None, 224)               36064     
                                                                 
 dense_3 (Dense)             (None, 32)                7200      
                                                                 
 dense_4 (Dense)             (None, 320)               10560     
                                                                 
 dense_5 (Dense)             (None, 128)               41088     
                                                                 
 dense_6 (Dense)             (None, 1)                 1

In [25]:
tuned_model.evaluate(X_test, y_test)

11/11 [==============================] - 0s 3ms/step - loss: 34.9780 - mean_absolute_error: 34.9780


[34.977996826171875, 34.977996826171875]